# Visualizing Neo4j graphs in yFiles Graphs for Jupyter <a target="_blank" href="https://colab.research.google.com/github/yWorks/yfiles-jupyter-graphs/blob/main/examples/16_neo4j_import.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before using the graph widget, install all necessary packages.

In [ ]:
%pip install yfiles_jupyter_graphs --quiet
%pip install neo4j --quiet
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget

You can also open this notebook in Google Colab when Google Colab's custom widget manager is enabled:

In [ ]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

<a target="_blank" href="https://colab.research.google.com/github/yWorks/yfiles-jupyter-graphs/blob/main/examples/16_neo4j_import.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## How to import a graph
- either import the graph directly when initilizing: ```GraphWidget(graph=your_graph)```
- or use the ```w.import_graph(your_graph)``` function, if you already initilized a Widget called ```w```

### Helper function to directly show a Cypher query as graph

In [ ]:
NEO4J_URI      = "neo4j+ssc://demo.neo4jlabs.com" 
NEO4J_USERNAME = "movies"
NEO4J_PASSWORD = "movies"

# create a neo4j session to run queries
driver = GraphDatabase.driver(uri = NEO4J_URI, auth = (NEO4J_USERNAME, NEO4J_PASSWORD), database = 'movies')
session = driver.session()

# directly show the graph resulting from the given Cypher query
def showGraph(cypher: str):
    widget = GraphWidget(graph = session.run(cypher).graph()) 
    display(widget)
    return widget

### Notes about Neo4j importer
- By default, the widget displays the node's label and the edge's type as text. This may be adjusted by setting a different label-mapping, e.g., see [02_label_mapping](./02_label_mapping.ipynb).
- Node labels are combined with ':' and added as 'label' property on the item
- Node and relationship properties are available on the graph item's `properties`.
- Each node label or relationship type is assigned a specific color.

## Use Cypher to display a graph

In [ ]:
w = showGraph("MATCH (s)-[r]->(t) RETURN s,r,t LIMIT 20")

# Map data to visualization

The yFiles Graphs for Jupyter widget provides different mapping function with which we can display node and edge properties in the diagram.

To access the 'properties' data, you can use the data key in squared brackets: ```['properties']['key'] ```

Possible node keys in this example are 'born', 'name', 'tagline', 'votes', 'title', 'released' and 'label' 

To visualize all properties, we remove any additional node data except the properties

In [ ]:
properties = [node['properties'] for node in w.nodes]
formattedProperties = ''.join(f"{node}\n" for node in properties)
print(formattedProperties)

In [ ]:
w2 = GraphWidget(graph = session.run("MATCH (s)-[r]->(t) RETURN s,r,t LIMIT 20").graph())

def custom_node_label_mapping(index, node):
    """let the label be the name or the title"""
    properties = node.get('properties', {})
    return properties.get('title', properties.get('name', 'no label'));
w2.node_label_mapping = custom_node_label_mapping

def custom_relationship_label_mapping(index, node):
    """let the label be the role"""
    properties = node.get('properties', {})
    return properties.get('roles', ['no role'])[0];
w2.edge_label_mapping = custom_relationship_label_mapping

w2.hierarchic_layout()
display(w2)

An alternative method for changing the names and titles to labels, utilizing the "title" and "name" properties stored in the node properties:

In [ ]:
w3 = GraphWidget(graph = session.run("MATCH (s)-[r]->(t) RETURN s,r,t LIMIT 20").graph())

w3.node_label_mapping = lambda node: node['properties']['name'] if 'name' in node['properties'] else node['properties']['title']

w3.hierarchic_layout()

w3.set_heat_mapping(lambda element: element['properties']['votes']/ 7000 if 'votes' in element['properties'] else 0)

display(w3)